In [1]:
import os

os.environ["KERAS_BACKEND"] = "jax"

import sys

import keras
import numpy as np

sys.path.append("../")
import bayesflow as bf

In [2]:
def brownian_motion(theta):
    num_params = theta.shape[1]
    sigma = np.eye(num_params) * 0.5
    t_max = 3.0
    n_t = 20
    h = 1.0
    dt = 1.0 * t_max / n_t
    batch_size = theta.shape[0]
    y = np.empty((batch_size, n_t + 1, num_params))

    # standard Brownian motion
    dW = np.random.multivariate_normal(np.zeros(num_params), np.eye(num_params) * h, (batch_size, n_t)) * np.sqrt(dt)
    W = np.cumsum(dW, axis=1)
    W = np.insert(W, 0, np.zeros(num_params), axis=1)
    time_steps = np.linspace(0.0, t_max, n_t + 1)

    for i in range(batch_size):
        drift_values = np.outer(time_steps, theta[i])
        y[i] = drift_values + sigma.dot(W[i].T).T
    time_encoding = np.tile(time_steps / t_max, [batch_size, 1])
    return np.concatenate([y, time_encoding[..., np.newaxis]], axis=-1)

In [3]:
def multi_simulator(batch_size, num_params=2, n_x=10, epsilon_x=0.10):
    theta = np.random.multivariate_normal(np.zeros(num_params), np.eye(num_params), size=(batch_size,))

    x = np.array(
        [
            np.random.multivariate_normal(theta[i], epsilon_x * np.eye(num_params), size=(n_x,))
            for i in range(batch_size)
        ]
    )

    y = brownian_motion(theta)

    return {"theta": theta, "x": x, "y": y}

In [4]:
multi_simulator(10)

{'theta': array([[-0.36016717, -1.26176628],
        [-1.15021267, -0.10449944],
        [ 1.64221116, -0.90553591],
        [ 1.28555812,  1.69952892],
        [-0.0560067 , -0.85958031],
        [ 2.11984816,  0.59070642],
        [ 1.17256363,  1.37418888],
        [ 0.29287948,  0.38485871],
        [ 0.35739706, -0.58582126],
        [-0.09067404,  0.51611574]]),
 'x': array([[[-0.18446187, -1.07931862],
         [-0.45358078, -0.89179915],
         [-0.59403116, -1.68467241],
         [-0.23079961, -0.95542636],
         [-0.59200474, -1.16782998],
         [-0.53818183, -1.06515746],
         [-0.02249932, -1.71264346],
         [-0.25234235, -1.66102794],
         [-0.86116503, -0.83133967],
         [-0.37129663, -0.65327702]],
 
        [[-1.0849818 , -0.09907775],
         [-1.26847849, -0.37642071],
         [-1.31328862, -0.19238341],
         [-1.39755153, -0.14545645],
         [-1.27547567, -0.23867304],
         [-1.42908666,  0.38465374],
         [-1.65664109,  0.353

In [5]:
train_data = bf.datasets.OfflineDataset(multi_simulator(2048), batch_size=64)
val_data = bf.datasets.OfflineDataset(multi_simulator(512), batch_size=64)

/Users/marvin/miniforge3/envs/bayesflow/lib/python3.11/site-packages/keras/src/backend/jax/numpy.py:262: UserWarning: Explicitly requested dtype int64 requested in arange is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return jnp.arange(start, stop, step=step, dtype=dtype)
/Users/marvin/miniforge3/envs/bayesflow/lib/python3.11/site-packages/keras/src/backend/jax/numpy.py:262: UserWarning: Explicitly requested dtype int64 requested in arange is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return jnp.arange(start, stop, step=step, dtype=dtype)


In [6]:
inference_network = bf.networks.CouplingFlow()

In [7]:
summary_network_x = bf.networks.DeepSet()
summary_network_y = bf.networks.LSTNet()

fusion_summary_network = bf.networks.fusion.LateFusionSummaryNetwork({"x": summary_network_x, "y": summary_network_y})

In [8]:
approximator = bf.approximators.Approximator(
    inference_network=inference_network,
    summary_network=fusion_summary_network,
    configurator=bf.configurators.DictConfigurator(inference_variables=["theta"], summary_variables=["x", "y"]),
)

In [9]:
optimizer = keras.optimizers.AdamW(
    learning_rate=1e-4,
    global_clipnorm=1.0,
)

In [10]:
approximator.compile(optimizer=optimizer)

In [11]:
fit_history = approximator.fit(train_data, epochs=100, validation_data=val_data)

AttributeError: 'dict' object has no attribute 'shape'